# 🥱 LangChain篇_實作 Plan-and-execute-Agent

在人工智能領域中，"計劃與執行"（Plan-and-Execute）代理是一種強大的工具，能夠通過先制定計劃，再執行子任務的方式來完成複雜目標。這種方法的靈感主要來源於 [BabyAGI](https://github.com/yoheinakajima/babyagi) 項目和 ["Plan-and-Solve"](https://arxiv.org/abs/2305.04091). 論文。本文將帶您深入了解這種智能代理的工作原理和實現方法。

核心概念
- 計劃階段：通常由大型語言模型（LLM）完成，負責制定整體策略和子任務。
- 執行階段：由配備各種工具的獨立代理執行具體任務。

❤️ Created by [hengshiousheu](https://huggingface.co/Heng666).

# 實現步驟

## 安裝套件包

在本演練中，

我們將使用以下套件：

In [ ]:
%pip install --upgrade --quiet langchain
%pip install --upgrade --quiet langchain-openai
%pip install --upgrade --quiet langchain_community
%pip install --upgrade --quiet langchain_experimental
%pip install --upgrade --quiet duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 28.3 MB/s eta 0:00:00


## 設置環境變數

為了使用 OpenAI API，我們需要設置 OPENAI_API_KEY 環境變數。在 Colab 中，我們可以使用 userdata 來安全地存儲和訪問 API 密鑰。

In [ ]:
import os
import getpass
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

> 安全提示：請確保不要直接在代碼中暴露您的 API 密鑰。使用 Colab 的 userdata 功能是一種安全的做法。

## 1. 導入必要的庫

首先，我們需要導入實現計劃與執行代理所需的各種庫和工具：


In [ ]:
from langchain.chains import LLMMathChain
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_core.tools import Tool
from langchain_experimental.plan_and_execute import (
    PlanAndExecute,
    load_agent_executor,
    load_chat_planner,
)
from langchain_openai import ChatOpenAI, OpenAI

## 2. 配置工具
在計劃與執行代理系統中，工具扮演著至關重要的角色。它們就像是代理的"手"和"眼睛"，使其能夠與外界互動並執行各種任務。接下來，我們設置 Agent 可以使用的工具，包括搜索引擎和計算機：

In [ ]:
search = DuckDuckGoSearchAPIWrapper()
llm = OpenAI(temperature=0)
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
    ),
]

> 提示：選擇合適的工具對於代理的性能至關重要。根據任務需求，可以添加或自定義更多工具。

## 3. 設置計劃者、執行者和代理
完成工具配置後，下一步是設置計劃者（Planner）、執行者（Executor）和最終的代理（Agent）。這三個組件共同構成了計劃與執行代理系統的核心架構


In [ ]:
model = ChatOpenAI(temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor)

## 4.運行示例
讓我們通過一個實際例子來測試我們的計劃與執行代理：

In [ ]:
agent.run(
    "高雄的五十嵐飲料店一杯珍奶多少錢？如果要五杯總共多少？最後回應記得用繁體中文"
)



> Entering new AgentExecutor chain...
Thought: The current objective is to confirm the price of pearl milk tea at 50 Arashi Beverage Store.

Action:
```
{
  "action": "Search",
  "action_input": "Price of pearl milk tea at 50 Arashi Beverage Store"
}
```
Observation: Understanding Pearl Tea As I took my first sip of pearl tea, I couldn't help but wonder about the intricate process that goes into creating this renowned beverage. The blend of tea, milk, and chewy tapioca pearls seemed simple yet complex, hinting at a story waiting to be unraveled. Bubble Tea for Coffee Lovers. 1. Coffee Milk Tea. Coffee enthusiasts, rejoice! Gong Cha's Coffee Milk Tea is a delightful fusion of coffee and milk tea, offering the best of both worlds in one cup. 2. Caramel Macchiato. Indulge in the velvety goodness of the Caramel Macchiato. Every major city in the Philippines is home to an Infinitea Milktea Menu Prices In Philippines outlet. Delicious pearl milk tea and other cool beverages like lemonade c

'單杯珍珠奶茶的價格在五十嵐飲料店約為TWD40 - 60（約USD1.3 - 1.95）每杯。五杯珍珠奶茶的總價格為TWD200（約USD6.5）。'

這個任務要求代理完成兩個子任務：

1. 找看看高雄五十嵐手搖飲料店的珍珠奶茶價格
2. 計算珍珠奶茶五杯時的售價

代理將首先制定計劃，然後逐步執行每個子任務，最終得出完整答案。

## 執行過程

### 任務一: 查詢珍珠奶茶價格

1. **初始搜索**
   - 代理首先搜索「Price of pearl milk tea at 50 Arashi Beverage Store」
   - 結果: 獲得了一些關於珍珠奶茶的一般信息,但沒有特定價格

2. **精確搜索**
   - 代理進行了多次更精確的搜索,試圖找到特定價格
   - 結果: 未能找到五十嵐飲料店的確切價格

3. **結論**
   - 代理建議直接聯繫店家或實地訪問以獲取準確價格

### 任務二: 計算總價

1. **價格假設**
   - 代理假設單杯價格為 TWD40

2. **計算過程**
   - 使用計算器工具: `40 * 5`
   - 結果: 200

3. **結論**
   - 五杯珍珠奶茶的總價為 TWD200 (約 USD6.5)

## 最終回答

代理綜合了搜索結果和計算,給出了以下回答:

『單杯珍珠奶茶的價格在五十嵐飲料店約為 TWD40 - 60（約 USD1.3 - 1.95）每杯。五杯珍珠奶茶的總價格為 TWD200（約 USD6.5）。』


## 結論
計劃與執行代理展示了人工智能在處理複雜、多步驟任務時的強大能力。通過將任務分解為可管理的子任務，並利用專門的工具，這種方法能夠高效地解決各種問題。
